In [ ]:
%%capture
%load_ext kamu
%import_dataset matches
%import_dataset player-scores

In [ ]:
%%capture
%%sql -o scores -q
select
    *
from `player-scores`

In [ ]:
%%capture
%%sql -o leaderboard -q

with best_scores as (
    select
        name,
        max(score) as best_score
    from `player-scores`
    where not is_bot
    group by name
),

last_scores as (
    select name, score as last_score from (
        select
            name,
            row_number() over (partition by name order by event_time desc) as rn,
            score
        from `player-scores`
        where not is_bot
    )
    where rn = 1    
)

select
    * 
from best_scores b
join last_scores l
on b.name = l.name

In [ ]:
%%capture
%%sql -o last_match -q

select
    * 
from matches
order by event_time desc
limit 1

In [ ]:
%%local
import os
import pandas as pd
import hvplot.pandas
import holoviews as hv
from matplotlib import pyplot as plt
from IPython.display import display, HTML

from matplotlib import pyplot as plt
from classes.process import PreProcess, Video, Visualize

import time
from datetime import datetime, timezone
import panel as pn
import holoviews.plotting.bokeh
from bokeh.plotting import figure

import json

_ = plt.ioff()

# this script reads the RL replay file and parses it into a json(internally) using rattletrap
# afterwards the necessary data is extracted and then turned into a heatmap, displays the match in 2d; see specific explanation below
# the main player is the player from which the replay was grabbed

proc = PreProcess()
heat = Visualize()

#load the converted json file from dataset
queried_replay = json.loads(last_match.loc[0].replay_json)

#process the json for car/bal location and other required variables
car_positions = proc.get_paths(queried_replay)

#extract relevant data from the prior process into format we use for visualization
car_histories = proc.car_positions_to_histories(car_positions[0])

#grab var needed in other class
teamord, player_ids, player_nam, teamnr, vip = proc.get_var()

#turn data into heatmaps, stats etc..
os.makedirs("output", exist_ok=True)
fig = heat.pcolormesh(car_histories, scores, teamord, player_ids)
plt.axis("off")
plt.title("Player location density (RAW)")
plt.savefig("output/pcolormesh.png", bbox_inches='tight')
plt.close()

fig = heat.pcolor(car_histories, scores, teamord, player_ids)
plt.axis("off")
plt.title("Player location density (GB)")
plt.savefig("output/pcolor.png", bbox_inches='tight')
plt.close()

video = Video(car_histories,player_nam,teamord,player_ids,vip,teamnr)
html_video = video.render(max_duration=30.0, fps=24, width_inches=5)
# HACK: In Chrome video will not autoplay unless muted
html_video = html_video.replace("autoplay", "muted autoplay", 1)
plt.close()

In [ ]:
%%local
dt = last_match.loc[0].event_time.replace(tzinfo=timezone.utc).astimezone(tz=None)

display(HTML('<h1>ODDS ecoSystem: E-Sports Data Pipeline Demo</h1>'))

pn.Row(
    pn.Column(
        pn.pane.HTML(f"<h4>Latest Match: <strong>{dt.date()} {dt.time()}</strong></h4>"),
        pn.pane.HTML(html_video),
    ),
    pn.Column(
        pn.panel("output/pcolormesh.png"),
        pn.panel("output/pcolor.png"),
    ),
    pn.Column(
        leaderboard.hvplot.table(
            columns=["name", "best_score", "last_score"], 
            selectable=True,
            width=500,
            title="Best Scores",
        ),
        scores.hvplot.box(
            "score",
            by="name",
            height=500,
            width=500,
            title="Player score distribution",
        )
    ),
)

In [ ]:
%%local
display(HTML('''
<script>
setTimeout(() => {
  window.location.reload();
}, 30000); // refresh every 30s
</script>
'''))